In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import random
import string
import xlwt
from xlwt import Workbook

In [2]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import callbacks

In [3]:
symbols = '0123456789'
num_symbols = len(symbols)
img_shape = (50,135,1)
print(num_symbols)

10


In [4]:
# in this block we difine our model
def create_model():
    img = layers.Input(shape=img_shape) 
    # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 5 branches from it.
    #Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [5]:
# remove lines from image by dilation && closing && blur
#def image_manipulation(dir):
    # reading the image from file
    #img = cv2.imread(dir,0) 
    # we need a kernel
    #kernel = np.ones((3,3),np.uint8)
    # dilate
    #dilate = cv2.morphologyEx(img,cv2.MORPH_DILATE,kernel)
    # close
    #close = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernel)
    # bilateral on close an dilate for 2 times
    #blur_close = cv2.bilateralFilter(close,9,75,75)
    #blur_dilate = cv2.bilateralFilter(dilate,9,75,75)
    #blur2_dilate = cv2.bilateralFilter(blur_dilate,9,75,75)
    #blur2_close = cv2.bilateralFilter(blur_close,9,75,75)
    # original image
    #cv2.imshow('image',img)
    #cv2.imshow('close',close)
    #cv2.imshow('dilate',dilate)
    #cv2.imshow('blur2_close',blur2_close)
    #cv2.imshow('blur2_dilate',blur2_dilate)
    #cv2.imshow('thresh on blur2_close',thresh)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #return blur2_close,blur2_dilate

In [18]:
# saving those manipulated images
#a = os.listdir()
#b = os.listdir('captcha')
#for i in range(0,len(b)):
    #c , d = image_manipulation('captcha/'+ b[i])
    #cv2.imwrite('blur2close/'+'_blur2close_'+b[i],c)
    #cv2.imwrite('blur2dilate/'+'_blur2dilate_'+b[i],d)

In [5]:
# using blur2close + blur2dilate + captcha instead of just captcha
def preprocess_data():
    n_samples = len(os.listdir('All'))
    X = np.zeros((n_samples, 50, 135, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir('All')):
        # Read image as grayscale
        img = cv2.imread(os.path.join('All/', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            # Scale and reshape image
            img = img / 255.0
            img = np.reshape(img, (50, 135, 1))
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]

In [6]:
model=create_model();
model.summary();

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 135, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 135, 16)  160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 25, 68, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 68, 32)   4640        max_pooling2d[0][0]              
______________________________________________________________________________________________

In [7]:
#model = create_model()
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=30,verbose=1, validation_split=0.2)

Train on 134 samples, validate on 34 samples
Epoch 1/30
134/134 [==============================] - 5s 39ms/sample - loss: 5.1529 - dense_1_loss: 0.9622 - dense_3_loss: 1.0213 - dense_5_loss: 0.9927 - dense_7_loss: 0.9497 - dense_9_loss: 1.0256 - dense_1_accuracy: 0.1418 - dense_3_accuracy: 0.0821 - dense_5_accuracy: 0.0746 - dense_7_accuracy: 0.2015 - dense_9_accuracy: 0.2164 - val_loss: 0.0000e+00 - val_dense_1_loss: 0.0000e+00 - val_dense_3_loss: 0.0000e+00 - val_dense_5_loss: 0.0000e+00 - val_dense_7_loss: 0.0000e+00 - val_dense_9_loss: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00 - val_dense_3_accuracy: 0.0000e+00 - val_dense_5_accuracy: 0.0000e+00 - val_dense_7_accuracy: 1.0000 - val_dense_9_accuracy: 0.0000e+00
Epoch 2/30
134/134 [==============================] - 0s 2ms/sample - loss: 4.7354 - dense_1_loss: 0.9270 - dense_3_loss: 0.9209 - dense_5_loss: 0.9148 - dense_7_loss: 0.8748 - dense_9_loss: 0.9343 - dense_1_accuracy: 0.1940 - dense_3_accuracy: 0.0746 - dense_5_accuracy: 

Epoch 13/30
134/134 [==============================] - 0s 2ms/sample - loss: 2.4631 - dense_1_loss: 0.5527 - dense_3_loss: 0.5559 - dense_5_loss: 0.4916 - dense_7_loss: 0.4069 - dense_9_loss: 0.5687 - dense_1_accuracy: 0.2612 - dense_3_accuracy: 0.2537 - dense_5_accuracy: 0.2313 - dense_7_accuracy: 0.2910 - dense_9_accuracy: 0.2239 - val_loss: 0.0000e+00 - val_dense_1_loss: 0.0000e+00 - val_dense_3_loss: 0.0000e+00 - val_dense_5_loss: 0.0000e+00 - val_dense_7_loss: 0.0000e+00 - val_dense_9_loss: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00 - val_dense_3_accuracy: 0.0000e+00 - val_dense_5_accuracy: 0.0000e+00 - val_dense_7_accuracy: 0.0000e+00 - val_dense_9_accuracy: 0.0000e+00
Epoch 14/30
134/134 [==============================] - 0s 2ms/sample - loss: 2.1397 - dense_1_loss: 0.4341 - dense_3_loss: 0.5343 - dense_5_loss: 0.4342 - dense_7_loss: 0.2793 - dense_9_loss: 0.4298 - dense_1_accuracy: 0.2761 - dense_3_accuracy: 0.2761 - dense_5_accuracy: 0.2985 - dense_7_accuracy: 0.3060 - dens

Epoch 25/30
134/134 [==============================] - 0s 3ms/sample - loss: 0.8349 - dense_1_loss: 0.2273 - dense_3_loss: 0.1385 - dense_5_loss: 0.1820 - dense_7_loss: 0.0993 - dense_9_loss: 0.1661 - dense_1_accuracy: 0.3731 - dense_3_accuracy: 0.4030 - dense_5_accuracy: 0.3881 - dense_7_accuracy: 0.4030 - dense_9_accuracy: 0.3881 - val_loss: 0.0000e+00 - val_dense_1_loss: 0.0000e+00 - val_dense_3_loss: 0.0000e+00 - val_dense_5_loss: 0.0000e+00 - val_dense_7_loss: 0.0000e+00 - val_dense_9_loss: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00 - val_dense_3_accuracy: 0.0000e+00 - val_dense_5_accuracy: 0.0000e+00 - val_dense_7_accuracy: 0.0000e+00 - val_dense_9_accuracy: 0.0000e+00
Epoch 26/30
134/134 [==============================] - 0s 3ms/sample - loss: 0.7040 - dense_1_loss: 0.1480 - dense_3_loss: 0.1462 - dense_5_loss: 0.1330 - dense_7_loss: 0.1218 - dense_9_loss: 0.1306 - dense_1_accuracy: 0.3806 - dense_3_accuracy: 0.3881 - dense_5_accuracy: 0.3657 - dense_7_accuracy: 0.3806 - dens

In [8]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 10))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt#, sum(probs) / 5

In [23]:
wb = Workbook()
sheet1 = wb.add_sheet('Sheet 1')
sheet1.write(0, 0, 'name of the JPEG file')
sheet1.write(0, 1, 'predicted value')
sheet1.write(0, 2, 'true/false')
a = os.listdir('captcha')
positive = 0
negetive = 0
for i in range(0,56):
    print(f"filename:{a[i]} , result is : ",predict('captcha/'+a[i]))
    # row start at 1 , column at 0
    sheet1.write(i+1, 0, a[i])
    sheet1.write(i+1, 1, predict('captcha/'+a[i])) 
    if a[i][0:5] == (predict('captcha/'+a[i])):
        print('true')
        positive = positive + 1
        sheet1.write(i+1, 2, 'true')
    else:
        print('false')
        negetive = negetive + 1
        sheet1.write(i+1, 2, 'false')

filename:13469.jpg , result is :  13469
true
filename:13963.jpg , result is :  13963
true
filename:16723.jpg , result is :  16723
true
filename:18641.jpg , result is :  18641
true
filename:18955.jpg , result is :  18955
true
filename:21118.jpg , result is :  21118
true
filename:21987.jpg , result is :  21987
true
filename:24184.jpg , result is :  24184
true
filename:26971.jpg , result is :  26971
true
filename:27114.jpg , result is :  27114
true
filename:28192.jpg , result is :  28192
true
filename:33261.jpg , result is :  33261
true
filename:34425.jpg , result is :  34425
true
filename:35125.jpg , result is :  35125
true
filename:35165.jpg , result is :  35165
true
filename:36248.jpg , result is :  36248
true
filename:37341.jpg , result is :  37341
true
filename:37826.jpg , result is :  37826
true
filename:37917.jpg , result is :  37917
true
filename:39621.jpg , result is :  39621
true
filename:39872.jpg , result is :  39872
true
filename:42445.jpg , result is :  42445
true
filename:4

In [24]:
# see the number of true and false cases
print('true cases: ',positive,' false cases : ',negetive)

true cases:  56  false cases :  0


In [25]:
# saving the excel file
wb.save('xlswt result.xls')

In [26]:
# open the result file
os.startfile("xlswt result.xls")